In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import matplotlib
import control as ct

# Tenta usar um backend interativo
try:
    matplotlib.use('TkAgg') # Ou 'Qt5Agg', 'GTK3Agg'
except ImportError:
    print("Backend TkAgg não encontrado, usando o padrão.")
except Exception as e: # Captura outros erros potenciais de backend
    print(f"Erro ao configurar o backend TkAgg: {e}, usando o padrão.")


class CableRobotTrajectory:

    def __init__(self, H=265, L=320, r_car=10, offset=0, T_total=10):
        # Parâmetros do Robô
        self.H = H
        self.L = L
        self.r_car = r_car
        self.offset = offset
        self.anchor_Ex = 0 + self.offset
        self.anchor_Ey = self.H
        self.anchor_Dx = self.L - self.offset
        self.anchor_Dy = self.H
        self.T_total = T_total # Tempo total padrão para a trajetória

        # --- PARÂMETROS FÍSICOS ---
        self.m_efetuador = 0.12  # Massa do efetuador em kg
        self.g = 9.81          # Aceleração da gravidade em m/s^2
        self.alpha_plano = np.deg2rad(80) 

        # Atributos da Trajetória da Animação (a serem preenchidos pelos métodos generate_*)
        self.x_anim_pts = np.array([])
        self.y_anim_pts = np.array([])
        self.t_anim_pts = np.array([]) # Tempos correspondentes a cada ponto da animação
        self.num_anim_frames = 0 

        self.x_main_pts = np.array([]) # Pontos X principais da trajetória
        self.y_main_pts = np.array([]) # Pontos Y principais da trajetória
        self.t_main_pts = np.array([]) # Tempos dos pontos principais

        # Atributos Cinemáticos da Trajetória da Animação
        self.compE_anim = np.array([])
        self.compD_anim = np.array([])
        self.angleE_anim = np.array([])
        self.angleD_anim = np.array([])
        self.velLinE_anim = np.array([])
        self.velLinD_anim = np.array([])

        # Perfis normalizados GLOBAIS para a trajetória da animação
        self.f_profile_anim = np.array([]) # Fator de progresso normalizado para toda a animação
        self.pos_profile_anim = np.array([]) # Posição normalizada global (pode não ser o _posProfile simples)
        self.vel_profile_anim = np.array([]) # Velocidade normalizada global
        self.acc_profile_anim = np.array([]) # Aceleração normalizada global
        
        self.initial_compE_for_angle = 0 
        self.initial_compD_for_angle = 0

        self.default_R_traj = 80
        self.default_xc_traj = 120
        self.default_yc_traj = 120

        # Atributos para dados de referência das plots estáticas
        self.xTraj_continuous_ref_for_plot = np.array([])
        self.yTraj_continuous_ref_for_plot = np.array([])
        self.t_smooth_for_plot = np.array([]) 
        self.compE_ref_plot = np.array([])
        self.compD_ref_plot = np.array([])
        self.angleE_ref_plot = np.array([])
        self.angleD_ref_plot = np.array([])
        self.velLinE_ref_plot = np.array([])
        self.velLinD_ref_plot = np.array([])
        
        # Handles Gráficos para animação
        self.fig_anim = None
        self.ax_anim_xy, self.ax_comp, self.ax_angle = None, None, None
        self.effector_scatter, self.line_cableE, self.line_cableD, self.line_trajectory_trace = None, None, None, None
        self.line_compE_anim_plot, self.line_compD_anim_plot, self.marker_comp = None, None, None
        self.line_angleE_anim_plot, self.line_angleD_anim_plot, self.marker_angle = None, None, None

        # Histórico para plots dinâmicos na animação
        self.hist_time_anim, self.hist_compE_anim, self.hist_compD_anim = [], [], []
        self.hist_angleE_anim, self.hist_angleD_anim = [], []
        self.trajectory_coords_x_hist, self.trajectory_coords_y_hist = [], []
        self.plot_idx_anim = 0

        self.pause_duration_ms = 30 
        self.save_video_flag = False
        self.video_filename = 'robot_animation.mp4'
        self._calculate_reference_kinematics_for_plots()

        self.x_actual_anim = np.array([])  # Posição X REAL do efetuador
        self.y_actual_anim = np.array([])  # Posição Y REAL do efetuador

        self.angleE_actual_anim = np.array([]) # Ângulo REAL do motor E
        self.angleD_actual_anim = np.array([]) # Ângulo REAL do motor D

        self.compE_actual_anim = np.array([]) # Comprimento REAL do cabo E
        self.compD_actual_anim = np.array([]) # Comprimento REAL do cabo D
        self.t_actual_anim = np.array([])

    def _posProfile(self, f):
        f_clipped = np.clip(f, 0, 1)
        return 35*f_clipped**4 - 84*f_clipped**5 + 70*f_clipped**6 - 20*f_clipped**7

    def _generate_interpolated_trajectory(self, x_main, y_main, frames_between_main_pts, 
                                      segmented_motion=False, approach_duration=2.0):
        if len(x_main) < 2:
            # Lógica para trajetória de ponto único...
            self.x_anim_pts, self.y_anim_pts = np.copy(x_main), np.copy(y_main)
            self.t_anim_pts = np.zeros_like(x_main, dtype=float)
            self.num_anim_frames = len(x_main)
            self._calculate_kinematics(); return

        self.x_main_pts, self.y_main_pts = x_main, y_main

        # --- LÓGICA UNIFICADA DE CÁLCULO DE TEMPO  ---
        segment_distances = np.sqrt(np.diff(x_main)**2 + np.diff(y_main)**2)
        starts_at_origin = np.allclose(x_main[0], 0) and np.allclose(y_main[0], 0)

        if starts_at_origin and len(segment_distances) > 1:
            # Caso especial: trajetória parte da origem, controla a velocidade da aproximação.
            duration_approach = min(approach_duration, self.T_total)
            time_for_rest = self.T_total - duration_approach
            rest_of_distances = segment_distances[1:]
            total_rest_distance = np.sum(rest_of_distances)
            if total_rest_distance > 1e-6 and time_for_rest > 0:
                durations_rest = (rest_of_distances / total_rest_distance) * time_for_rest
            else:
                durations_rest = np.zeros_like(rest_of_distances)
            segment_durations = np.concatenate(([duration_approach], durations_rest))
        else:
            # Lógica padrão: distribui o tempo total pelo comprimento de todos os segmentos.
            total_path_length = np.sum(segment_distances)
            if total_path_length > 1e-6:
                segment_durations = (segment_distances / total_path_length) * self.T_total
            else:
                segment_durations = np.zeros_like(segment_distances)
        
        # Vetor de tempo para os pontos principais, agora sempre correto.
        t_main_correct = np.concatenate(([0], np.cumsum(segment_durations)))

        # --- LÓGICA UNIFICADA DE INTERPOLACÃO (LOOP ÚNICO) ---
        x_list, y_list, t_list = [], [], []
        num_segments = len(x_main) - 1

        for i in range(num_segments):
            x1, y1, t1 = x_main[i], y_main[i], t_main_correct[i]
            x2, y2, t2 = x_main[i+1], y_main[i+1], t_main_correct[i+1]

            f_linear = np.linspace(0, 1, frames_between_main_pts + 1)
            
            # A única diferença entre os modos é o perfil de interpolação 'alpha'
            if segmented_motion:
                # Para movimento "para e vai", usa o perfil S-Curve
                alpha = self._posProfile(f_linear)
            else:
                # Para movimento contínuo, usa um perfil linear (uma rampa)
                alpha = f_linear

            # Interpola Posição (com o perfil 'alpha' escolhido) e Tempo (sempre linear)
            x_segment = x1 + alpha * (x2 - x1)
            y_segment = y1 + alpha * (y2 - y1)
            t_segment = t1 + f_linear * (t2 - t1)

            # Adiciona os pontos do segmento, evitando duplicatas
            x_list.extend(x_segment[:-1]); y_list.extend(y_segment[:-1]); t_list.extend(t_segment[:-1])
        
        # Adiciona o último ponto da trajetória
        x_list.append(x_main[-1]); y_list.append(y_main[-1]); t_list.append(t_main_correct[-1])

        self.x_anim_pts, self.y_anim_pts, self.t_anim_pts = np.array(x_list), np.array(y_list), np.array(t_list)
        self.num_anim_frames = len(self.x_anim_pts)

        self._calculate_kinematics()

    def _calculate_time_from_distance(self, x_main, y_main):
        """
        Calcula o vetor de tempo (t_main) para uma trajetória com base na distância
        real entre os pontos, garantindo uma velocidade de percurso constante.
        """
        if len(x_main) < 2:
            # Se houver menos de 2 pontos, a duração é simples.
            return np.array([0.0] * len(x_main))

        # Calcular a distância percorrida em cada ponto da trajetória principal
        distances = np.sqrt(np.diff(x_main)**2 + np.diff(y_main)**2)
        cumulative_dist = np.concatenate(([0], np.cumsum(distances)))
        
        total_distance = cumulative_dist[-1]
        
        if total_distance > 1e-6:
            dist_based_progress = cumulative_dist / total_distance
        else:
            # Fallback se a distância total for zero 
            dist_based_progress = np.linspace(0, 1, len(x_main))

        # Usar a progressão baseada na distância para aplicar o perfil de movimento
        t_main = self.T_total * self._posProfile(dist_based_progress)
        return t_main

    def _find_closest_entry_and_prepare_path(self, x_shape_pts, y_shape_pts):
        """
        Encontra o ponto em uma forma mais próximo da origem (0,0),
        reorganiza os pontos para começar dali e adiciona a origem como o ponto inicial.
        """
        # Calcula a distância de cada ponto da forma até a origem
        dist_from_origin = np.sqrt(x_shape_pts**2 + y_shape_pts**2)
        
        # Encontra o índice do ponto mais próximo
        closest_idx = np.argmin(dist_from_origin)
        
        x_rolled = np.roll(x_shape_pts, -closest_idx)
        y_rolled = np.roll(y_shape_pts, -closest_idx)
        
        # Garante que a trajetória da forma seja um loop fechado, adicionando o primeiro ponto ao final
        x_looped = np.append(x_rolled, x_rolled[0])
        y_looped = np.append(y_rolled, y_rolled[0])
        
        # Adiciona a origem (0,0) como o ponto de partida de toda a trajetória
        x_final = np.concatenate(([0.0], x_looped))
        y_final = np.concatenate(([0.0], y_looped))
        
        return x_final, y_final

    def generate_circular_trajectory(self, R_traj=None, xc_traj=None, yc_traj=None,
                                 N_pts_main=30, frames_between_main_pts=20,
                                 start_from_origin=False, segmented_motion=True, 
                                 approach_duration=2.0):
        """
        Gera os pontos principais de uma trajetória circular.
        A lógica de tempo e interpolação é gerenciada pela função _generate_interpolated_trajectory.
        """
        # Define os parâmetros da forma
        _R = R_traj if R_traj is not None else self.default_R_traj
        _xc = xc_traj if xc_traj is not None else self.default_xc_traj
        _yc = yc_traj if yc_traj is not None else self.default_yc_traj
        if N_pts_main < 2: N_pts_main = 2
        
        # Gera os pontos que definem a geometria do círculo
        raw_angles_main = np.linspace(0, 2 * np.pi, N_pts_main, endpoint=True) 
        x_shape_pts = _xc + _R * np.cos(raw_angles_main)
        y_shape_pts = _yc + _R * np.sin(raw_angles_main)
        
        #  Prepara a lista de pontos principais (x_main, y_main)
        if start_from_origin:
            x_main, y_main = self._find_closest_entry_and_prepare_path(x_shape_pts, y_shape_pts)
        else:
            x_main = x_shape_pts
            y_main = y_shape_pts
        
        self._generate_interpolated_trajectory(x_main, y_main, frames_between_main_pts, 
                                            segmented_motion=segmented_motion, 
                                            approach_duration=approach_duration)

        self._calculate_reference_kinematics_for_plots(shape='custom', x_custom=x_main, y_custom=y_main)

    def generate_square_trajectory(self, side_length_mm=150, center_x=120, center_y=120,
                               N_total_main_pts_on_perimeter=20, frames_between_main_pts=5,
                               start_from_origin=False, segmented_motion=True):
        s, cx, cy = side_length_mm, center_x, center_y
        x_ur, y_ur = cx+s/2, cy+s/2; x_ul, y_ul = cx-s/2, cy+s/2
        x_ll, y_ll = cx-s/2, cy-s/2; x_lr, y_lr = cx+s/2, cy-s/2
        
        corners = np.array([[x_ur, y_ur], [x_ul, y_ul], [x_ll, y_ll], [x_lr, y_lr], [x_ur, y_ur]])
        
        if N_total_main_pts_on_perimeter < 4: N_total_main_pts_on_perimeter = 4
        
        # Lógica para gerar os pontos da forma 
        dense_x_perimeter, dense_y_perimeter = [], []
        points_per_dense_side = max(2, int(200 / 4))
        for i in range(4):
            x_s = np.linspace(corners[i,0], corners[i+1,0], points_per_dense_side, endpoint=(i==3))
            y_s = np.linspace(corners[i,1], corners[i+1,1], points_per_dense_side, endpoint=(i==3))
            if i < 3:
                dense_x_perimeter.extend(x_s[:-1]); dense_y_perimeter.extend(y_s[:-1])
            else:
                dense_x_perimeter.extend(x_s); dense_y_perimeter.extend(y_s)
        
        dense_x_perimeter = np.array(dense_x_perimeter)
        dense_y_perimeter = np.array(dense_y_perimeter)
        distances = np.sqrt(np.diff(dense_x_perimeter)**2 + np.diff(dense_y_perimeter)**2)
        cumulative_distances = np.concatenate(([0], np.cumsum(distances)))
        total_perimeter_length = cumulative_distances[-1]
        target_distances = np.linspace(0, total_perimeter_length, N_total_main_pts_on_perimeter, endpoint=True)
        x_shape_pts = np.interp(target_distances, cumulative_distances, dense_x_perimeter)
        y_shape_pts = np.interp(target_distances, cumulative_distances, dense_y_perimeter)

        if len(x_shape_pts) == 0:
            print("Erro: Nenhum ponto principal gerado para a trajetória quadrada.")
            self._generate_interpolated_trajectory([],[],frames_between_main_pts)
            return

        if start_from_origin:
            x_main, y_main = self._find_closest_entry_and_prepare_path(x_shape_pts, y_shape_pts)
        else:
            x_main = np.append(x_shape_pts, x_shape_pts[0])
            y_main = np.append(y_shape_pts, y_shape_pts[0])

        self._generate_interpolated_trajectory(x_main, y_main, frames_between_main_pts)
        self._calculate_reference_kinematics_for_plots(shape='custom',x_custom=x_main,y_custom=y_main)
    
    def generate_triangle_trajectory(self, side_length_mm=150, center_x=None, center_y=None,
                                 N_total_main_pts_on_perimeter=30, frames_between_main_pts=5,
                                 start_from_origin=False, cluster_at_vertices=False, 
                                 segmented_motion=True, approach_duration=2.0):
        """
        Gera os pontos principais de uma trajetória triangular.
        A lógica de tempo e interpolação é gerenciada pela função _generate_interpolated_trajectory.
        """
        # Define os parâmetros da forma
        cx = center_x if center_x is not None else self.default_xc_traj
        cy = center_y if center_y is not None else self.default_yc_traj
        s = side_length_mm

        # Gera os pontos que definem a geometria do triângulo
        altura = s * np.sqrt(3) / 2
        v1 = (cx, cy + (2/3) * altura)
        v2 = (cx + s/2, cy - (1/3) * altura)
        v3 = (cx - s/2, cy - (1/3) * altura)
        vertices = [v1, v2, v3]
        pts_per_side = N_total_main_pts_on_perimeter // 3
        x_shape_pts, y_shape_pts = [], []
        for i in range(3):
            start_vertex, end_vertex = vertices[i], vertices[(i + 1) % 3]
            if not cluster_at_vertices:
                progress = np.linspace(0, 1, pts_per_side, endpoint=False)
            else:
                linear_progress = np.linspace(0, 1, pts_per_side, endpoint=False)
                progress = 0.5 - 0.5 * np.cos(linear_progress * np.pi)
            x_shape_pts.extend(start_vertex[0] + (end_vertex[0] - start_vertex[0]) * progress)
            y_shape_pts.extend(start_vertex[1] + (end_vertex[1] - start_vertex[1]) * progress)
        x_shape_pts, y_shape_pts = np.array(x_shape_pts), np.array(y_shape_pts)

        # Prepara a lista de pontos principais (x_main, y_main)
        if start_from_origin:
            x_main, y_main = self._find_closest_entry_and_prepare_path(x_shape_pts, y_shape_pts)
        else:
            x_main = np.append(x_shape_pts, x_shape_pts[0])
            y_main = np.append(y_shape_pts, y_shape_pts[0])

        self._generate_interpolated_trajectory(x_main, y_main, frames_between_main_pts, 
                                            segmented_motion=segmented_motion, 
                                            approach_duration=approach_duration)
        
        # Atualiza os gráficos de referência 
        self._calculate_reference_kinematics_for_plots(shape='custom', x_custom=x_main, y_custom=y_main)
        
    def _calculate_kinematics(self):
        if self.num_anim_frames == 0:
            attrs_to_clear = ['compE_anim','compD_anim','angleE_anim','angleD_anim',
                                'velLinE_anim','velLinD_anim','f_normalized_profile',
                                'pos_normalized_profile','vel_normalized_profile','acc_normalized_profile']
            for attr in attrs_to_clear: setattr(self, attr, np.array([]))
            self.initial_compE_for_angle, self.initial_compD_for_angle = 0,0
            return
            
        self.compE_anim = np.sqrt((self.anchor_Ey - self.y_anim_pts)**2 + (self.anchor_Ex - self.x_anim_pts)**2)
        self.compD_anim = np.sqrt((self.anchor_Dy - self.y_anim_pts)**2 + (self.anchor_Dx - self.x_anim_pts)**2)
        
        if self.num_anim_frames > 0:
            self.initial_compE_for_angle = self.compE_anim[0]
            self.initial_compD_for_angle = self.compD_anim[0]
        else:
            self.initial_compE_for_angle, self.initial_compD_for_angle = 0,0
            
        self.angleE_anim = (self.compE_anim - self.initial_compE_for_angle) * 180 / (np.pi * self.r_car)
        self.angleD_anim = (self.compD_anim - self.initial_compD_for_angle) * 180 / (np.pi * self.r_car)
        self.f_normalized_profile = np.linspace(0, 1, self.num_anim_frames) if self.num_anim_frames > 0 else np.array([])
        
        if len(self.f_normalized_profile) > 0:
            self.pos_normalized_profile = self._posProfile(self.f_normalized_profile)
            
            if self.num_anim_frames > 1:
                h_f_norm = self.f_normalized_profile[1] - self.f_normalized_profile[0]
                h_f_norm = h_f_norm if abs(h_f_norm) > 1e-9 else 1e-9
                raw_vel_normalized = np.gradient(self.pos_normalized_profile, h_f_norm)
                raw_acc_normalized = np.gradient(raw_vel_normalized, h_f_norm)
                max_abs_vel = np.max(np.abs(raw_vel_normalized))
                self.vel_normalized_profile = raw_vel_normalized / max_abs_vel if max_abs_vel > 1e-9 else raw_vel_normalized
                max_abs_acc = np.max(np.abs(raw_acc_normalized))
                self.acc_normalized_profile = raw_acc_normalized / max_abs_acc if max_abs_acc > 1e-9 else raw_acc_normalized
            else:
                self.vel_normalized_profile = np.array([0.0]); self.acc_normalized_profile = np.array([0.0])
        else:
            self.pos_normalized_profile, self.vel_normalized_profile, self.acc_normalized_profile = np.array([]),np.array([]),np.array([])
            
        self.velLinE_anim = np.zeros_like(self.compE_anim)
        self.velLinD_anim = np.zeros_like(self.compD_anim)
        
        if self.num_anim_frames > 1 and len(self.t_anim_pts) == self.num_anim_frames:
            dt_check = np.diff(self.t_anim_pts)
            is_monotonic_time = np.all(dt_check > 1e-9) 
            if is_monotonic_time:
                try:
                    self.velLinE_anim = np.gradient(self.compE_anim, self.t_anim_pts, edge_order=2 if self.num_anim_frames > 2 else 1)
                    self.velLinD_anim = np.gradient(self.compD_anim, self.t_anim_pts, edge_order=2 if self.num_anim_frames > 2 else 1)
                except ValueError: is_monotonic_time = False
            if not is_monotonic_time: 
                print("Aviso: Usando cálculo manual de diferenças finitas para velocidades lineares dos cabos (animação).")
                for i in range(self.num_anim_frames):
                    if i == 0: 
                        dt = self.t_anim_pts[1]-self.t_anim_pts[0] if self.num_anim_frames > 1 else 1e-6
                        dt_safe = (np.sign(dt) if dt != 0 else 1) * 1e-6 if abs(dt) < 1e-9 else dt
                        self.velLinE_anim[i]=(self.compE_anim[1]-self.compE_anim[0])/dt_safe if self.num_anim_frames > 1 else 0
                        self.velLinD_anim[i]=(self.compD_anim[1]-self.compD_anim[0])/dt_safe if self.num_anim_frames > 1 else 0
                    elif i == self.num_anim_frames - 1: 
                        dt = self.t_anim_pts[i]-self.t_anim_pts[i-1] if self.num_anim_frames > 1 else 1e-6
                        dt_safe = (np.sign(dt) if dt != 0 else 1) * 1e-6 if abs(dt) < 1e-9 else dt
                        self.velLinE_anim[i]=(self.compE_anim[i]-self.compE_anim[i-1])/dt_safe if self.num_anim_frames > 1 else 0
                        self.velLinD_anim[i]=(self.compD_anim[i]-self.compD_anim[i-1])/dt_safe if self.num_anim_frames > 1 else 0
                    else: 
                        dt = self.t_anim_pts[i+1]-self.t_anim_pts[i-1]
                        dt_safe = (np.sign(dt) if dt != 0 else 1) * 1e-6 if abs(dt) < 1e-9 else dt
                        self.velLinE_anim[i]=(self.compE_anim[i+1]-self.compE_anim[i-1])/dt_safe
                        self.velLinD_anim[i]=(self.compD_anim[i+1]-self.compD_anim[i-1])/dt_safe
        elif self.num_anim_frames == 1: 
            self.velLinE_anim = np.array([0.0]); self.velLinD_anim = np.array([0.0])

        if self.num_anim_frames > 1:
            self.velLinE_anim[0] = 0.0
            self.velLinD_anim[0] = 0.0

    def _calculate_reference_kinematics_for_plots(self, N_smooth_ref=250, shape='circle', **kwargs):

        if N_smooth_ref < 2 : N_smooth_ref = 2 
        if shape == 'circle':
            _R = kwargs.get('R', self.default_R_traj); _xc = kwargs.get('xc', self.default_xc_traj)
            _yc = kwargs.get('yc', self.default_yc_traj); f_s_ref = np.linspace(0,1,N_smooth_ref)
            self.t_smooth_for_plot = f_s_ref*self.T_total; n_p_prof_ref = self._posProfile(f_s_ref)
            theta_ref = 2*np.pi*n_p_prof_ref
            self.xTraj_continuous_ref_for_plot = _xc+_R*np.cos(theta_ref)
            self.yTraj_continuous_ref_for_plot = _yc+_R*np.sin(theta_ref)
        elif shape == 'custom':
            self.xTraj_continuous_ref_for_plot=kwargs.get('x_custom',np.array([]))
            self.yTraj_continuous_ref_for_plot=kwargs.get('y_custom',np.array([]))
            self.t_smooth_for_plot=kwargs.get('t_custom',np.array([]))
            if len(self.t_smooth_for_plot)==0 and len(self.xTraj_continuous_ref_for_plot)>0:
                 self.t_smooth_for_plot=np.linspace(0,self.T_total,len(self.xTraj_continuous_ref_for_plot))
        else: # Default to circle
            f_s_ref=np.linspace(0,1,N_smooth_ref); self.t_smooth_for_plot=f_s_ref*self.T_total
            n_p_prof_ref=self._posProfile(f_s_ref); theta_ref=2*np.pi*n_p_prof_ref
            self.xTraj_continuous_ref_for_plot=self.default_xc_traj+self.default_R_traj*np.cos(theta_ref)
            self.yTraj_continuous_ref_for_plot=self.default_yc_traj+self.default_R_traj*np.sin(theta_ref)
        if len(self.xTraj_continuous_ref_for_plot) > 1 and len(self.t_smooth_for_plot) == len(self.xTraj_continuous_ref_for_plot):
            self.compE_ref_plot = np.sqrt((self.anchor_Ey-self.yTraj_continuous_ref_for_plot)**2+(self.anchor_Ex-self.xTraj_continuous_ref_for_plot)**2)
            self.compD_ref_plot = np.sqrt((self.anchor_Dy-self.yTraj_continuous_ref_for_plot)**2+(self.anchor_Dx-self.xTraj_continuous_ref_for_plot)**2)
            self.angleE_ref_plot = (self.compE_ref_plot-self.compE_ref_plot[0])*180/(np.pi*self.r_car)
            self.angleD_ref_plot = (self.compD_ref_plot-self.compD_ref_plot[0])*180/(np.pi*self.r_car)
            self.velLinE_ref_plot = np.zeros_like(self.compE_ref_plot)
            self.velLinD_ref_plot = np.zeros_like(self.compD_ref_plot)
            use_manual_diff_ref = False
            if len(self.t_smooth_for_plot) > 1:
                dt_check_ref = np.diff(self.t_smooth_for_plot)
                if np.any(np.abs(dt_check_ref) < 1e-9): use_manual_diff_ref = True
            else: use_manual_diff_ref = True
            if not use_manual_diff_ref and len(self.t_smooth_for_plot) >= 2 : 
                try:
                    self.velLinE_ref_plot = np.gradient(self.compE_ref_plot, self.t_smooth_for_plot, edge_order=1)
                    self.velLinD_ref_plot = np.gradient(self.compD_ref_plot, self.t_smooth_for_plot, edge_order=1)
                except ValueError: use_manual_diff_ref = True
            if use_manual_diff_ref and len(self.t_smooth_for_plot) > 1: 
                 for i in range(len(self.t_smooth_for_plot)):
                    if i == 0 : dt = self.t_smooth_for_plot[1]-self.t_smooth_for_plot[0]
                    elif i == len(self.t_smooth_for_plot)-1: dt = self.t_smooth_for_plot[i]-self.t_smooth_for_plot[i-1]
                    else: dt = self.t_smooth_for_plot[i+1]-self.t_smooth_for_plot[i-1]
                    dt_safe = (np.sign(dt) if dt!=0 else 1) *1e-6 if abs(dt)<1e-9 else dt
                    if i == 0:
                        self.velLinE_ref_plot[i]=(self.compE_ref_plot[1]-self.compE_ref_plot[0])/dt_safe
                        self.velLinD_ref_plot[i]=(self.compD_ref_plot[1]-self.compD_ref_plot[0])/dt_safe
                    elif i == len(self.t_smooth_for_plot)-1:
                        self.velLinE_ref_plot[i]=(self.compE_ref_plot[i]-self.compE_ref_plot[i-1])/dt_safe
                        self.velLinD_ref_plot[i]=(self.compD_ref_plot[i]-self.compD_ref_plot[i-1])/dt_safe
                    else:
                        self.velLinE_ref_plot[i]=(self.compE_ref_plot[i+1]-self.compE_ref_plot[i-1])/dt_safe
                        self.velLinD_ref_plot[i]=(self.compD_ref_plot[i+1]-self.compD_ref_plot[i-1])/dt_safe
            elif len(self.t_smooth_for_plot) == 1: 
                self.velLinE_ref_plot[0]=0; self.velLinD_ref_plot[0]=0
        else:
            attrs_to_clear_ref = ['compE_ref_plot','compD_ref_plot','angleE_ref_plot','angleD_ref_plot',
                                  'velLinE_ref_plot','velLinD_ref_plot','xTraj_continuous_ref_for_plot',
                                  'yTraj_continuous_ref_for_plot','t_smooth_for_plot']
            for attr in attrs_to_clear_ref: setattr(self, attr, np.array([]))

    def export_to_gcode(self, filename="trajetoria.gcode", feed_rate=1500.0):
        """
        Exporta a trajetória da animação (self.x_anim_pts, self.y_anim_pts)
        para um arquivo G-Code.
        """
        if self.num_anim_frames == 0:
            print("Nenhuma trajetória foi gerada para exportar.")
            return

        line_number = 1
        with open(filename, 'w') as f:
            print(f"Exportando {self.num_anim_frames} pontos para {filename}...")
            
            # Move para a posição inicial com G00 (movimento rápido)
            initial_x = self.x_anim_pts[0]
            initial_y = self.y_anim_pts[0]
            f.write(f"N{line_number:03d} G00 X{initial_x:.3f} Y{initial_y:.3f}\n")
            line_number += 1
            
            # Percorre os pontos restantes com G01 (movimento linear) com taxa de avanço
            for i in range(1, self.num_anim_frames):
                x = self.x_anim_pts[i]
                y = self.y_anim_pts[i]
                f.write(f"N{line_number:03d} G01 X{x:.3f} Y{y:.3f} F{feed_rate:.1f}\n")
                line_number += 1
            
            # Adiciona comando de fim de programa
            f.write(f"N{line_number:03d} M30\n")
        
        print("Exportação concluída com sucesso.")

    def plot_normalized_profiles(self):
     
        if len(self.f_normalized_profile) == 0: print("Gere trajetória primeiro."); return
        fig, ax = plt.subplots(figsize=(10,7)); fig.canvas.manager.set_window_title('Perfis Normalizados (Anim. Traj.)')
        ax.plot(self.f_normalized_profile,self.pos_normalized_profile,'b-',label='Posição Norm. (Global)')
        ax.plot(self.f_normalized_profile,self.vel_normalized_profile,'r-',label='Velocidade Norm. (Global)')
        ax.plot(self.f_normalized_profile,self.acc_normalized_profile,'g-',label='Aceleração Norm. (Global)')
        ax.set_xlabel('f (Progresso Normalizado da Animação)'); ax.set_ylabel('Valor Normalizado')
        ax.set_title('Perfis de Movimento Normalizados Globais (Baseado na Trajetória da Animação)')
        ax.legend(loc='best'); ax.grid(True); plt.tight_layout(); plt.show(block=False)

    def plot_xy_trajectory(self):
        
        if len(self.x_anim_pts)==0: print("Gere trajetória primeiro."); return
        fig, ax = plt.subplots(figsize=(8,8)); fig.canvas.manager.set_window_title('Trajetória XY')
        if len(self.xTraj_continuous_ref_for_plot)>0:
            ax.plot(self.xTraj_continuous_ref_for_plot,self.yTraj_continuous_ref_for_plot,'k--',linewidth=1.0,label='Trajetória de Referência')
        ax.plot(self.x_anim_pts,self.y_anim_pts,'b.-',linewidth=1,markersize=3,label='Trajetória da Animação')
        if len(self.x_main_pts)>0: 
            ax.plot(self.x_main_pts,self.y_main_pts,'ro',markerfacecolor='none',markersize=7,label='Pontos Principais')
        ax.set_aspect('equal',adjustable='box'); plot_margin=20
        all_x_elements = [arr for arr in [self.x_anim_pts, self.x_main_pts, [self.anchor_Ex, self.anchor_Dx], self.xTraj_continuous_ref_for_plot] if arr is not None and len(arr) > 0]
        all_y_elements = [arr for arr in [self.y_anim_pts, self.y_main_pts, [self.anchor_Ey, self.anchor_Dy], self.yTraj_continuous_ref_for_plot] if arr is not None and len(arr) > 0]
        if all_x_elements and all_y_elements:
            all_x_for_lim = np.concatenate(all_x_elements); all_y_for_lim = np.concatenate(all_y_elements)
            if len(all_x_for_lim) > 0 and len(all_y_for_lim) > 0:
                ax.set_xlim(all_x_for_lim.min()-plot_margin,all_x_for_lim.max()+plot_margin)
                ax.set_ylim(all_y_for_lim.min()-plot_margin,all_y_for_lim.max()+plot_margin)
        ax.set_xlabel('x (mm)'); ax.set_ylabel('y (mm)'); ax.set_title('Trajetória XY')
        ax.legend(loc='best'); ax.grid(True); plt.show(block=False)
        
    def plot_cable_data(self):
        
        if len(self.t_smooth_for_plot)==0 or len(self.compE_ref_plot)==0: 
            print("Dados de referência não calculados. Chamando _calculate_reference_kinematics_for_plots...")
            self._calculate_reference_kinematics_for_plots() 
        if len(self.t_anim_pts)==0 or len(self.compE_anim)==0: 
            print("Gere uma trajetória e calcule a cinemática da animação primeiro."); return
        fig,axs=plt.subplots(3,1,sharex=True,figsize=(10,12)); fig.canvas.manager.set_window_title('Dados Cabos (Anim vs Ref)')
        xlim_val = self.T_total if self.T_total > 0 else (self.t_anim_pts.max() if len(self.t_anim_pts)>0 else 1)
        if len(self.t_smooth_for_plot)>0: axs[0].plot(self.t_smooth_for_plot,self.compE_ref_plot,'r:',label='Ref. Comp. E')
        if len(self.t_smooth_for_plot)>0: axs[0].plot(self.t_smooth_for_plot,self.compD_ref_plot,'g:',label='Ref. Comp. D')
        axs[0].plot(self.t_anim_pts,self.compE_anim,'r-',label='Anim. Comp. E')
        axs[0].plot(self.t_anim_pts,self.compD_anim,'g-',label='Anim. Comp. D')
        axs[0].set_ylabel('Comprimento (mm)'); axs[0].set_title('Comprimento Cabos'); axs[0].legend(loc='best'); axs[0].grid(True); axs[0].set_xlim(0,xlim_val)
        if len(self.t_smooth_for_plot)>0: axs[1].plot(self.t_smooth_for_plot,self.velLinE_ref_plot,'r:',label='Ref. Vel. E')
        if len(self.t_smooth_for_plot)>0: axs[1].plot(self.t_smooth_for_plot,self.velLinD_ref_plot,'g:',label='Ref. Vel. D')
        axs[1].plot(self.t_anim_pts,self.velLinE_anim,'r--',label='Anim. Vel. E')
        axs[1].plot(self.t_anim_pts,self.velLinD_anim,'g--',label='Anim. Vel. D')
        axs[1].set_ylabel('Velocidade (mm/s)'); axs[1].set_title('Velocidade Linear Cabos'); axs[1].legend(loc='best'); axs[1].grid(True); axs[1].set_xlim(0,xlim_val)
        if len(self.t_smooth_for_plot)>0: axs[2].plot(self.t_smooth_for_plot,self.angleE_ref_plot,'r:',label='Ref. Ângulo E')
        if len(self.t_smooth_for_plot)>0: axs[2].plot(self.t_smooth_for_plot,self.angleD_ref_plot,'g:',label='Ref. Ângulo D')
        axs[2].plot(self.t_anim_pts,self.angleE_anim,'r-',label='Anim. Ângulo E')
        axs[2].plot(self.t_anim_pts,self.angleD_anim,'g-',label='Anim. Ângulo D')
        axs[2].set_xlabel('Tempo (s)'); axs[2].set_ylabel('Ângulo (°)'); axs[2].set_title('Ângulo Rotação Motores')
        axs[2].legend(loc='best'); axs[2].grid(True); axs[2].set_xlim(0,xlim_val)
        plt.tight_layout(); plt.show(block=False)

    def _setup_animation_figure(self):
        self.fig_anim = plt.figure(figsize=(12,8)); 
        self.fig_anim.canvas.manager.set_window_title('Anim. Robô & Dados')
        try: 
            manager = plt.get_current_fig_manager()
            if hasattr(manager, 'window'):
                if hasattr(manager.window, 'showMaximized'): manager.window.showMaximized()
                elif hasattr(manager.window, 'state') and hasattr(manager.window, 'zoomed'): manager.window.state('zoomed')
            elif hasattr(manager, 'frame') and hasattr(manager.frame, 'Maximize'): manager.frame.Maximize(True)
        except Exception: print("Aviso: Não foi possível maximizar janela automaticamente.")
        gs = self.fig_anim.add_gridspec(2,3,width_ratios=[3,1,1],height_ratios=[1,1], wspace=0.35, hspace=0.55) 
        self.ax_anim_xy = self.fig_anim.add_subplot(gs[:,0]); self.ax_comp = self.fig_anim.add_subplot(gs[0,1:])
        self.ax_angle = self.fig_anim.add_subplot(gs[1,1:])
        self.ax_anim_xy.set_aspect('equal',adjustable='box'); plot_margin=20
        valid_x_list = [arr for arr in [self.x_anim_pts, self.x_main_pts, [self.anchor_Ex, self.anchor_Dx], self.xTraj_continuous_ref_for_plot] if arr is not None and len(arr)>0]
        valid_y_list = [arr for arr in [self.y_anim_pts, self.y_main_pts, [self.anchor_Ey, self.anchor_Dy], self.yTraj_continuous_ref_for_plot] if arr is not None and len(arr)>0]
        if hasattr(self, 'default_yc_traj') and hasattr(self, 'default_R_traj'):
            valid_y_list.append(np.array([self.default_yc_traj - self.default_R_traj, self.default_yc_traj + self.default_R_traj]))
        if valid_x_list and valid_y_list:
            all_valid_x = np.concatenate(valid_x_list); all_valid_y = np.concatenate(valid_y_list)
            if len(all_valid_x) > 0 and len(all_valid_y) > 0:
                 self.ax_anim_xy.set_xlim(all_valid_x.min()-plot_margin,all_valid_x.max()+plot_margin)
                 self.ax_anim_xy.set_ylim(all_valid_y.min()-plot_margin,all_valid_y.max()+plot_margin)
        self.ax_anim_xy.set_title('Animação Trajetória', fontsize=12); self.ax_anim_xy.set_xlabel('X (mm)', fontsize=10); self.ax_anim_xy.set_ylabel('Y (mm)', fontsize=10); self.ax_anim_xy.grid(True)
        if len(self.xTraj_continuous_ref_for_plot) > 0:
            self.ax_anim_xy.plot(self.xTraj_continuous_ref_for_plot, self.yTraj_continuous_ref_for_plot, 'k--', linewidth=1.0, label='Trajetória de Referência')
        if len(self.t_anim_pts)>0:
            self.ax_comp.plot(self.t_anim_pts,self.compE_anim,'r:',lw=1,label='E Traj. Completa'); self.ax_comp.plot(self.t_anim_pts,self.compD_anim,'g:',lw=1,label='D Traj. Completa')
            self.ax_angle.plot(self.t_anim_pts,self.angleE_anim,'r:',lw=1,label='E Traj. Completa'); self.ax_angle.plot(self.t_anim_pts,self.angleD_anim,'g:',lw=1,label='D Traj. Completa')
        self.line_compE_anim_plot, = self.ax_comp.plot([],[],'r-',lw=2,label='Comp.E Atual'); self.line_compD_anim_plot, = self.ax_comp.plot([],[],'g-',lw=2,label='Comp.D Atual')
        self.marker_comp, = self.ax_comp.plot([],[],'ko',mfc='k',ms=5,label='Atual'); self.ax_comp.set_ylabel('Compr. (mm)',fontsize=10); self.ax_comp.set_title('Comprimento Cabos',fontsize=12)
        self.ax_comp.legend(loc='lower center',bbox_to_anchor=(0.5,-0.5),ncol=3,fontsize='x-small'); self.ax_comp.grid(True); self.ax_comp.set_xlim(0,self.t_anim_pts.max() if len(self.t_anim_pts)>0 else self.T_total)
        if len(self.compE_anim)>0 and len(self.compD_anim)>0: self.ax_comp.set_ylim(np.min([self.compE_anim.min(),self.compD_anim.min()])-10,np.max([self.compE_anim.max(),self.compD_anim.max()])+10)
        self.line_angleE_anim_plot, = self.ax_angle.plot([],[],'r-',lw=2,label='Âng.E Atual'); self.line_angleD_anim_plot, = self.ax_angle.plot([],[],'g-',lw=2,label='Âng.D Atual')
        self.marker_angle, = self.ax_angle.plot([],[],'ko',mfc='k',ms=5,label='Atual'); self.ax_angle.set_xlabel('Tempo (s)',fontsize=10); self.ax_angle.set_ylabel('Ângulo (°)',fontsize=10); self.ax_angle.set_title('Ângulo Motores',fontsize=12)
        self.ax_angle.legend(loc='lower center',bbox_to_anchor=(0.5,-0.6),ncol=3,fontsize='x-small'); self.ax_angle.grid(True); self.ax_angle.set_xlim(0,self.t_anim_pts.max() if len(self.t_anim_pts)>0 else self.T_total)
        if len(self.angleE_anim)>0 and len(self.angleD_anim)>0: self.ax_angle.set_ylim(np.min([self.angleE_anim.min(),self.angleD_anim.min()])-50,np.max([self.angleE_anim.max(),self.angleD_anim.max()])+50)
        if len(self.x_main_pts)>0: self.ax_anim_xy.plot(self.x_main_pts,self.y_main_pts,'o',color=[0.7,0.7,0.7],ms=6,label='Pts-Alvo')
        self.ax_anim_xy.plot(self.anchor_Ex,self.anchor_Ey,'sk',mfc='k',ms=8,label='Anc.E'); self.ax_anim_xy.plot(self.anchor_Dx,self.anchor_Dy,'sk',mfc='k',ms=8,label='Anc.D')
        if self.num_anim_frames>0:
            self.effector_scatter=self.ax_anim_xy.scatter(self.x_anim_pts[0],self.y_anim_pts[0],s=80,c='blue',zorder=10,label='Efetuador')
            self.line_cableE, = self.ax_anim_xy.plot([self.anchor_Ex,self.x_anim_pts[0]],[self.anchor_Ey,self.y_anim_pts[0]],color='red',lw=1.5,label='CaboE')
            self.line_cableD, = self.ax_anim_xy.plot([self.anchor_Dx,self.x_anim_pts[0]],[self.anchor_Dy,self.y_anim_pts[0]],color='green',lw=1.5,label='CaboD')
            self.line_trajectory_trace, = self.ax_anim_xy.plot([],[],linestyle='-',color=[0.4,0.4,0.8],lw=2,label='Rastro')
        self.ax_anim_xy.legend(loc='lower center',ncol=5,bbox_to_anchor=(0.5,-0.2),fontsize='x-small') 
        self.fig_anim.tight_layout(rect=[0,0.08,0.98,0.93])

    def _init_animation_func_anim(self):
        self.plot_idx_anim = 0 
        if self.num_anim_frames == 0: return []
        self.plot_idx_anim += 1; idx = 0 
        t,x,y = self.t_anim_pts[idx],self.x_anim_pts[idx],self.y_anim_pts[idx]
        self.effector_scatter.set_offsets([x,y]); self.line_cableE.set_data([self.anchor_Ex,x],[self.anchor_Ey,y])
        self.line_cableD.set_data([self.anchor_Dx,x],[self.anchor_Dy,y])
        self.trajectory_coords_x_hist,self.trajectory_coords_y_hist=[x],[y]
        self.line_trajectory_trace.set_data(self.trajectory_coords_x_hist,self.trajectory_coords_y_hist)
        self.hist_time_anim,self.hist_compE_anim,self.hist_compD_anim=[t],[self.compE_anim[idx]],[self.compD_anim[idx]]
        self.hist_angleE_anim,self.hist_angleD_anim=[self.angleE_anim[idx]],[self.angleD_anim[idx]]
        self.line_compE_anim_plot.set_data(self.hist_time_anim,self.hist_compE_anim)
        self.line_compD_anim_plot.set_data(self.hist_time_anim,self.hist_compD_anim)
        self.marker_comp.set_data([t],[self.compE_anim[idx]])
        self.line_angleE_anim_plot.set_data(self.hist_time_anim,self.hist_angleE_anim)
        self.line_angleD_anim_plot.set_data(self.hist_time_anim,self.hist_angleD_anim)
        self.marker_angle.set_data([t],[self.angleE_anim[idx]])
        return (self.effector_scatter,self.line_cableE,self.line_cableD,self.line_trajectory_trace,
                self.line_compE_anim_plot,self.line_compD_anim_plot,self.marker_comp,
                self.line_angleE_anim_plot,self.line_angleD_anim_plot,self.marker_angle)

    def _update_animation_func_anim(self, frame_idx_from_func_anim):
        if self.num_anim_frames<=1: return self._init_animation_func_anim()
        current_data_idx = frame_idx_from_func_anim + 1 
        if current_data_idx >= self.num_anim_frames: 
            return (self.effector_scatter, self.line_cableE, self.line_cableD, self.line_trajectory_trace,
                self.line_compE_anim_plot, self.line_compD_anim_plot, self.marker_comp,
                self.line_angleE_anim_plot, self.line_angleD_anim_plot, self.marker_angle)
        self.plot_idx_anim +=1 
        x = self.x_anim_pts[current_data_idx]
        y = self.y_anim_pts[current_data_idx]
        t = self.t_anim_pts[current_data_idx]
        self.effector_scatter.set_offsets([x,y])
        self.line_cableE.set_data([self.anchor_Ex,x],[self.anchor_Ey,y])
        self.line_cableD.set_data([self.anchor_Dx,x],[self.anchor_Dy,y])
        self.trajectory_coords_x_hist.append(x)
        self.trajectory_coords_y_hist.append(y)
        self.line_trajectory_trace.set_data(self.trajectory_coords_x_hist,self.trajectory_coords_y_hist)
        self.hist_time_anim.append(t)
        self.hist_compE_anim.append(self.compE_anim[current_data_idx])
        self.hist_compD_anim.append(self.compD_anim[current_data_idx])
        self.hist_angleE_anim.append(self.angleE_anim[current_data_idx])
        self.hist_angleD_anim.append(self.angleD_anim[current_data_idx])
        self.line_compE_anim_plot.set_data(self.hist_time_anim,self.hist_compE_anim)
        self.line_compD_anim_plot.set_data(self.hist_time_anim,self.hist_compD_anim)
        self.marker_comp.set_data([t],[self.compE_anim[current_data_idx]])
        self.line_angleE_anim_plot.set_data(self.hist_time_anim,self.hist_angleE_anim)
        self.line_angleD_anim_plot.set_data(self.hist_time_anim,self.hist_angleD_anim)
        self.marker_angle.set_data([t],[self.angleE_anim[current_data_idx]])
        return (self.effector_scatter,self.line_cableE,self.line_cableD,self.line_trajectory_trace,
                self.line_compE_anim_plot,self.line_compD_anim_plot,self.marker_comp,
                self.line_angleE_anim_plot,self.line_angleD_anim_plot,self.marker_angle)

    def animate_trajectory(self, interval_ms=None, save_video=None, video_filename_override=None):
        gif_filename = "Trajetoria.gif"
        if not hasattr(self,'num_anim_frames') or self.num_anim_frames==0:
            if not hasattr(self,'x_anim_pts') or len(self.x_anim_pts)==0:
                 print("Trajetória vazia. Gere uma trajetória primeiro."); return
            else: self.num_anim_frames=len(self.x_anim_pts)
            if self.num_anim_frames==0: print("Trajetória ainda vazia. Abortando."); return
        self._setup_animation_figure()
        current_pause_ms=interval_ms if interval_ms is not None else self.pause_duration_ms
        current_save_flag=save_video if save_video is not None else self.save_video_flag
        current_video_filename=video_filename_override if video_filename_override is not None else self.video_filename
        print('Iniciando animação para visualização ao vivo...')
        frames_for_funcanim = self.num_anim_frames - 1 if self.num_anim_frames > 1 else 0
        ani=FuncAnimation(self.fig_anim,self._update_animation_func_anim,frames=frames_for_funcanim,
                            init_func=self._init_animation_func_anim,blit=True,
                            interval=current_pause_ms,repeat=False,save_count=None)
        print("Mostrando animação... Feche a janela da animação para prosseguir para o salvamento (se ativado).")
        plt.show() 
        if current_save_flag:
            if self.num_anim_frames == 0 : 
                print("Nenhum frame para salvar.")
            else:
                fps_video=1000/current_pause_ms if current_pause_ms>0 else 30; rounded_fps=max(1,round(fps_video))
                fn_mp4=current_video_filename; fn_mp4=fn_mp4 if fn_mp4.lower().endswith('.mp4') else fn_mp4+'.mp4'
                print(f"Tentando salvar MP4: {fn_mp4} (FPS: {rounded_fps})...")
                try:
                    ani.save(fn_mp4,writer='ffmpeg',fps=rounded_fps,dpi=150)
                    print('MP4 salvo.')
                except Exception as e_ffmpeg:
                    print(f"Falha FFMpeg: {e_ffmpeg}"); fn_gif=fn_mp4.replace('.mp4','.gif')
                    print(f"Tentando salvar GIF: {fn_gif} (FPS: {rounded_fps})...")
                    try: 
                        ani.save(gif_filename,writer='pillow',fps=rounded_fps,dpi=150)
                        print('GIF salvo.')
                    except Exception as e_pillow_gif: 
                        print(f"Falha Pillow: {e_pillow_gif}")
        print('Processo de animação concluído.')
    
    def _forward_kinematics(self, l_e, l_d):
        """
        Calcula a posição (x, y) do efetuador a partir dos comprimentos dos cabos l_e e l_d.
        """
        L = self.L
        H = self.H
        
        # Evita divisão por zero
        if L < 1e-6:
            return 0, 0
            
        # Fórmula da cinemática direta para este robô
        x = (l_e**2 - l_d**2 + L**2) / (2 * L)
        
        # Calcula y, garantindo que o argumento da raiz quadrada não seja negativo
        termo_raiz = l_e**2 - x**2
        termo_raiz[termo_raiz < 0] = 0 # Zera valores negativos por segurança numérica
        y = H - np.sqrt(termo_raiz)
        
        return x, y
    
    def run_dynamic_simulation(self):
        """
        Simula a resposta dinâmica, considerando a referência e a perturbação da carga estática.
        """
        if self.num_anim_frames == 0:
            print("Gere uma trajetória cinemática primeiro.")
            return

        print("Iniciando simulação dinâmica...")

        # --- Definir o Modelo do Sistema e Controlador PID ---
        Kp, Ki, Kd, Tf = 9.6187, 0.0180, 0.0052, 0.01
        s = ct.tf('s')
        G_pos = 2.953 / (0.004629 * s**2 + s)
        C_pid = ct.tf([Kd + Kp*Tf, Kp + Ki*Tf, Ki], [Tf, 1, 0])
        
        # --- Calcular a Carga Estática ---
        
        forca_no_cabo = (self.m_efetuador * self.g * np.sin(self.alpha_plano)) / 2
        tau_load = forca_no_cabo * (self.r_car / 1000) # Torque em N.m

        # --- Definir os modelos de malha fechada para Referência e Perturbação ---
        T_r = ct.feedback(C_pid * G_pos, 1)      # Ref -> Saída (y_ref)
        T_d = ct.feedback(G_pos, C_pid)          # Perturbação (torque_load) -> Saída (y_dist)
        
        # --- Preparar os Sinais para a Simulação ---
        t_sim = np.linspace(0, self.T_total, self.num_anim_frames)
        ref_angle_E_resampled = np.interp(t_sim, self.t_anim_pts, self.angleE_anim)
        ref_angle_D_resampled = np.interp(t_sim, self.t_anim_pts, self.angleD_anim)
        
        # A entrada de perturbação é um torque constante
        disturbance_input = tau_load * np.ones_like(t_sim)
        
        # --- Simular a resposta completa (Referência + Perturbação) ---
        print("Simulando resposta do motor E (com carga)...")
        _, y_ref_E = ct.forced_response(T_r, T=t_sim, U=ref_angle_E_resampled)
        _, y_dist_E = ct.forced_response(T_d, T=t_sim, U=disturbance_input)
        self.angleE_actual_anim = y_ref_E + y_dist_E # Resposta total

        print("Simulando resposta do motor D (com carga)...")
        _, y_ref_D = ct.forced_response(T_r, T=t_sim, U=ref_angle_D_resampled)
        _, y_dist_D = ct.forced_response(T_d, T=t_sim, U=disturbance_input)
        self.angleD_actual_anim = y_ref_D + y_dist_D # Resposta total

        self.t_actual_anim = t_sim

        # --- Calcular a Posição Real do Efetuador  ---
        print("Calculando posição real do efetuador via cinemática direta...")
        self.compE_actual_anim = (self.angleE_actual_anim * np.pi * self.r_car / 180) + self.initial_compE_for_angle
        self.compD_actual_anim = (self.angleD_actual_anim * np.pi * self.r_car / 180) + self.initial_compD_for_angle
        self.x_actual_anim, self.y_actual_anim = self._forward_kinematics(self.compE_actual_anim, self.compD_actual_anim)
        
        print("Simulação dinâmica concluída.")

    def plot_kinematic_vs_dynamic(self):
        """
        Plota um comparativo entre a resposta cinemática ideal e a dinâmica real.
        """
        if len(self.x_actual_anim) == 0:
            print("Execute a simulação dinâmica primeiro (run_dynamic_simulation).")
            return
            
        fig, axs = plt.subplots(3, 1, figsize=(12, 10), sharex=True)
        fig.suptitle('Comparativo: Cinemática Ideal vs. Dinâmica Real', fontsize=16)
        
        # Use o t_anim_pts para o ideal e t_actual_anim para o real
        t_ideal = self.t_anim_pts
        t_actual = self.t_actual_anim
        
        # Posição em X
        axs[0].plot(t_ideal, self.x_anim_pts, 'b--', label='X Ideal (Cinemático)')
        axs[0].plot(t_actual, self.x_actual_anim, 'b-', label='X Real (Dinâmico)')
        axs[0].set_ylabel('Posição X [mm]')
        axs[0].legend()
        axs[0].grid(True)
        
        # Posição em Y
        axs[1].plot(t_ideal, self.y_anim_pts, 'g--', label='Y Ideal (Cinemático)')
        axs[1].plot(t_actual, self.y_actual_anim, 'g-', label='Y Real (Dinâmico)')
        axs[1].set_ylabel('Posição Y [mm]')
        axs[1].legend()
        axs[1].grid(True)
        
        # Erro de Rastreamento
        x_ideal_resampled = np.interp(t_actual, t_ideal, self.x_anim_pts)
        y_ideal_resampled = np.interp(t_actual, t_ideal, self.y_anim_pts)
        error = np.sqrt((x_ideal_resampled - self.x_actual_anim)**2 + (y_ideal_resampled - self.y_actual_anim)**2)
        axs[2].plot(t_actual, error, 'r-', label='Erro ')
        axs[2].set_ylabel('Erro de Rastreamento [mm]')
        axs[2].set_xlabel('Tempo [s]')
        axs[2].legend()
        axs[2].grid(True)
        
        plt.tight_layout(rect=[0, 0, 1, 0.96])
        plt.show()
    
    def export_to_gcode(self, filename="trajetoria.gcode"):
        """
        Exporta a trajetória da animação (self.x_anim_pts, self.y_anim_pts)
        para um arquivo G-Code.
        """
        if self.num_anim_frames == 0:
            print("Nenhuma trajetória foi gerada para exportar.")
            return

        line_number = 1
        with open(filename, 'w') as f:
            print(f"Exportando {self.num_anim_frames} pontos para {filename}...")
            
            # Move para a posição inicial com G00 (movimento rápido)
            initial_x = self.x_anim_pts[0]
            initial_y = self.y_anim_pts[0]
            f.write(f"N{line_number:03d} G00 X{initial_x:.3f} Y{initial_y:.3f}\n")
            line_number += 1
            
            # Percorre os pontos restantes com G01 (movimento linear) com taxa de avanço
            for i in range(1, self.num_anim_frames):
                x = self.x_anim_pts[i]
                y = self.y_anim_pts[i]
                f.write(f"N{line_number:03d} G01 X{x:.3f} Y{y:.3f}\n")
                line_number += 1
            
            # Adiciona comando de fim de programa
            f.write(f"N{line_number:03d} M30\n")
        
        print("Exportação concluída com sucesso.")


# --- Exemplo de Uso --
if __name__ == '__main__':
    # Inicializa o robô
    robot = CableRobotTrajectory(T_total=10) 

    print("--- Gerando Trajetória Triangular Ideal ---")
    robot.generate_triangle_trajectory(
        side_length_mm=150, 
        N_total_main_pts_on_perimeter=21,
        frames_between_main_pts=10,
        start_from_origin=True,
        cluster_at_vertices=True,
        segmented_motion=False   
    )
    
    # Exporta o G-Code da trajetória do triângulo
    robot.export_to_gcode(filename="trajetoria_triangulo.gcode")

    robot.run_dynamic_simulation()

    robot.animate_trajectory(interval_ms=30)

    # --- PLOTS E ANÁLISES ---
    robot.plot_xy_trajectory()
    
    robot.plot_cable_data() 
    
    robot.plot_kinematic_vs_dynamic()

    print("\n--- Gerando Trajetória Circular Ideal ---")
    robot.generate_circular_trajectory(
        R_traj=80, 
        N_pts_main=21, 
        frames_between_main_pts=10,
        start_from_origin=True,
        segmented_motion=False 
    )

    # Exporta o G-Code
    robot.export_to_gcode(filename="trajetoria_circular.gcode")
    
    robot.run_dynamic_simulation()

    robot.animate_trajectory(interval_ms=30)

    # --- PLOTS E ANÁLISES ---
    robot.plot_xy_trajectory()
    
    robot.plot_cable_data() 
    
    robot.plot_kinematic_vs_dynamic()
    
   

--- Gerando Trajetória Triangular Ideal ---
Exportando 221 pontos para trajetoria_triangulo.gcode...
Exportação concluída com sucesso.
Iniciando simulação dinâmica...
Simulando resposta do motor E (com carga)...
Simulando resposta do motor D (com carga)...
Calculando posição real do efetuador via cinemática direta...
Simulação dinâmica concluída.


C:\Users\Fefon\AppData\Local\Temp\ipykernel_14304\2015625132.py:622: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  self.fig_anim.tight_layout(rect=[0,0.08,0.98,0.93])


Iniciando animação para visualização ao vivo...
Mostrando animação... Feche a janela da animação para prosseguir para o salvamento (se ativado).
Processo de animação concluído.

--- Gerando Trajetória Circular Ideal ---
Aviso: Usando cálculo manual de diferenças finitas para velocidades lineares dos cabos (animação).
Exportando 221 pontos para trajetoria_circular.gcode...
Exportação concluída com sucesso.
Iniciando simulação dinâmica...
Simulando resposta do motor E (com carga)...
Simulando resposta do motor D (com carga)...
Calculando posição real do efetuador via cinemática direta...
Simulação dinâmica concluída.
Iniciando animação para visualização ao vivo...
Mostrando animação... Feche a janela da animação para prosseguir para o salvamento (se ativado).
Processo de animação concluído.
